
اینجا صرفا کتابخانه ها و توابعی که در پروژه از آنها استفاده شده ایمپورت شده‌اند

In [7]:
import random
import collections
import heapq
from dataclasses import dataclass, field
from typing import List, Tuple, Callable, Any, Set
import numpy as np
from time import time
from tabulate import tabulate
import threading
import collections
from math import ceil
from enum import Enum
from functools import partial
from copy import deepcopy
import queue
from collections import deque
import collections


در اینجا کلاس ماتریس ها تعریف شده. برای هر یک تعدادی تابع تعریف شده مانند 
toggle: که صرفا مسئولیت تغییر حالت یک خانه و خانه‌های مجاورش را بر عهده میگیرد. متناظر با تابع جانشین که در مسائل سرچ باید تعریف شده باشد
is_solved: آیا به حالت نهایی که مدنظرمان بود که همه خانه ها صفر باشند رسیده‌ایم یا خیر؟
get_moves: تمامی اکشن های ممکنی که میتوانیم داشته باشیم. زدن کلید هر خانه از جدول...

state: وضعیت خانه‌های پازل ها که مجموعا دو به توان تعداد درایه‌هاست
initial state: پازل اولیه ای که ورودی داده میشود.
goal state: پازلی که همه خانه‌هایش صفر باشد به زبانی دیگر تابع ایز سولود آن مقدار درست دهد.

مسئله را به گرافی متناظر میکنیم. رئوس آن مجموعه استیت هاست و یال های بین آن همان عملیاتی است که باعث میشود دو استیت یا پازل به هم تبدیل شوند.

In [8]:
class LightsOutPuzzle:
    def __init__(self, board: List[List[int]]):
        self.board = np.array(board, dtype=np.uint8)
        self.size = len(board)

    def toggle(self, x, y):
        for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.board[nx, ny] ^= 1

    def is_solved(self):
        return np.all(self.board == 0)

    def get_moves(self):
        return [(x, y) for x in range(self.size) for y in range(self.size)]

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)


یک عنصر رندوم با خانه های صفر و یک از کلاس جدولی که بالا ساخته شده میسازد. 

In [9]:
def create_random_board(size: int,  seed: int, num_toggles: int = None):
    random.seed(time() if seed is None else seed)

    board = [[0 for _ in range(size)] for _ in range(size)]
    puzzle = LightsOutPuzzle(board)

    if num_toggles is None:
        num_toggles = random.randint(1, size * size)

    for _ in range(num_toggles):
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        puzzle.toggle(x, y)

    return puzzle.board

به ازای هر الگوریتمی که ما زده ایم چک میکند که آیا راه حلی پیدا شده یا نه؟ اگر پیدا شده اطلاعات مدنظر ما را میدهد. مثلا میگوید فلان مقدار عملیات انجام شده و آن عملیات، این مجموعه هستند.

In [10]:
def show_solution(
    puzzle: LightsOutPuzzle,
    solution: List[Tuple[int, int]],
    algorithm_name: str,
    nodes_visited: int,
    show_steps: bool = False,
):
    print(f"\nSolving with {algorithm_name}:")
    if solution is None:
        print("No solution found.")
        return
    print(f"Solution: {solution}")
    print(f"Nodes visited: {nodes_visited}")
    if show_steps:
        print("\nSolution steps:")
        for i, move in enumerate(solution):
            print(f"\nStep {i+1}: Toggle position {move}")
            puzzle.toggle(*move)
            print(puzzle)
        print(
            "\nFinal state (solved):"
            if puzzle.is_solved()
            else "\nFinal state (not solved):"
        )
        print(puzzle)

متناسب با نت بوک اولیه که در پروژه تحت اختیار ما قرار داده شده 9 جدول با پارامترهایی ساخته میشوند. 3 تا از آنها 3*3 هستند، 3 تا از آنها چهار در چهار و 3 تا پنج در پنج

In [12]:
tests = [
    create_random_board(3, 42),
    create_random_board(3, 41),
    create_random_board(3, 42, 5),
    create_random_board(4, 42),
    create_random_board(4, 41),
    create_random_board(4, 42, 5),
    create_random_board(5, 42),
    create_random_board(5, 41),
    create_random_board(5, 42, 5),
]

الگوریتم بی اف اس با استفاده از داده ساختار صف ساخته شده که میتوان با استفاده از آن اولین 
راسی که وارد صف شده را پردازش کنیم. رئوس مجاورش را بررسی کنیم و به صورت سطحی به هر راس کوتاه ترین مسیر را ارائه دهیم
برای هر راس کوتاه ترین مسیر از راس اولیه تا آن ذخیره میشودو مشخص میشود آیا اصلا آن راس قبلا ویزیت شده است یا خیر؟


In [15]:
def bfs_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    #states are puzzles
    #initial is input
    #successor is toggle
    #final is  when is_solved() = True

    initial = puzzle.board.tobytes()# for uniqueness
    bfsqueue = queue.Queue()
    bfsqueue.put((puzzle, []))
    visited = {initial}
    nodes_visited = 0

    while not bfsqueue.empty():
        prossesing_node, path = bfsqueue.get()#اولین نودی که وارد صف شده کیست و کوتاه ترین مسیر آن که ثبت شده چیست؟
        nodes_visited = nodes_visited + 1 #یک نود جدید دیدیم

        if prossesing_node.is_solved():#اگر به استیت نهایی رسیده ایم جواب را خروجی دهیم. مسیر وارد شدن به آن راس که میشود همان عملیات به ترتیب و همچنین تعداد عملیات
            return path, nodes_visited
        else:
          for move in prossesing_node.get_moves():
              new_node = deepcopy(prossesing_node)
              new_node.toggle(move[0], move[1])# پازل مجاور
              visit = new_node.board.tobytes()

              if visit not in visited:# اگر تا حالا دیده نشده پس کوتاه ترین مسیری که برای این راس جدید به آن رسیده ایم همین است.
                  visited.add(visit)
                  bfsqueue.put((new_node, path + [move]))

    return None, nodes_visited

    '''if solved == True:
      res = []
      while desitnation != 'None':
        res.append(father[desitnation])
        desitnation = father[desitnation]
        res.append(moves[desitnation])
      res.reverse()
      print(res)
      return res, len(res)
    return 'None' , 0
    '''







برای زدن الگوریتم 
ids
چون یک ترکیبی از 
dfs,bfs
با هم بود ابتدا یک تابع ساختم که با عمق محدود k
روی گراف حالات ما جستوجوی عمقی انجام دهد. هر بار تا جایی که جواب دهد محدودیت عمق را بیشتر میکنیم. به اصطلاح k
را یکی یکی اضافه میکنیم که اینکار در تابع ids
انجام میشود.


برای dfs
از داده ساختار پشته استفاده میشود چرا که تا جای ممکن روی پشته ماراس اضافه میکند و هر بار که به نقطه بدی برخورد کرد اخرین راس اضافه شده را از بالا بر میدارد و سراغ ادامه کار میرود


In [16]:
def dfs(puzzle: LightsOutPuzzle, k) -> Tuple[List[Tuple[int, int]], int]:
  #states are puzzles
  #initial is input
  #successor is toggle
  #final is  when is_solved() = True

  initial = puzzle.board.tobytes()# for uniqueness
  dfsstack = []
  dfsstack.append((puzzle, []))
  visited = {initial}
  nodes_visited = 0
  tool = 0# مسئولیت مدیریت عمق را دارد. اگر از محدودیت ما داشت فراتر میرفت جلوی آن را میگیرد.

  while not len(dfsstack)==0:# هر بار به هر مجاوری که جدید است و میتواند برود میرود به امید اینکه به جواب برسد.
    prossesing_node, path = dfsstack[len(dfsstack)-1]
    nodes_visited = nodes_visited + 1
    if prossesing_node.is_solved():
      return path, nodes_visited
    else:
      flag= True
      for move in prossesing_node.get_moves():
              new_node = deepcopy(prossesing_node)
              new_node.toggle(move[0], move[1])
              visit = new_node.board.tobytes()
              if visit not in visited and tool<k:
                  visited.add(visit)
                  dfsstack.append((new_node, path + [move]))
                  tool+=1
                  flag= False
                  break
      if flag:
        tool-=1
        dfsstack.pop()
  return None, nodes_visited



In [17]:
# TODO: Must return a list of tuples and the number of visited nodes
def ids_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    #states are puzzles
    #initial is input
    #successor is toggle
    #final is  when is_solved() = True

    k = 1
    sum = 0
    while (dfs(puzzle , k)[0] == None):# هر بار محدودیت عمق را یکی یکی اضافه میکنیم تا جایی که به اولین عمقی برسیم که جواب است. البته که تعداد نود های ویزیت شده هر بار جمع میشوند
        k+=1
        sum+= dfs(puzzle , k)[1]
        if k == 2**(puzzle.size**2):
          return None, 0


    return dfs(puzzle, k)[0], sum


فارغ از اینکه توابع ابتکاری ما چه هستند برای زدن کد astar
از داده ساختار heap
که یک جور صف اولویت روی کمترین مقدار است استفاده شده. چرا که ما هر بار میخواهیم به نودی برویم که کمترین میزان ممکن h(n)+g(n)
را دارد. و این از بین تمامی نود های موجودیست که ما داریم. پس باید یک جا اینها ذخیره شوند.
از جمله چالش هایی که بود این بود که به heap
بفهمانم کمترین یعنی چه؟ بر اساس چه ملاکی نود هارا ذخیره کند و آنها را مرتب کند تا در نهایت کمترین را خروجی دهد. اینجا بود که یک کلاس به نام 
setayi
درست شد و در آن مفهوم کمترین و بیشترین به صورت تابع 
__lt__, __eq__
تعریف شد. حال در کد الگوریتم از کلاس setayi
استفاده میکنیم.

In [18]:
class setayi:
  def __init__(self, adad, puz, pat):
    self.adad = adad
    self.puz = puz
    self.pat = pat
  def __lt__(self, other):
    return self.adad < other.adad
  def __eq__(self, other):
    return self.adad == other.adad


In [19]:
# TODO: Must return a list of tuples and the number of visited nodes
def astar_solve(puzzle: LightsOutPuzzle, heuristic: Callable[[LightsOutPuzzle], int]) -> Tuple[List[Tuple[int, int]], int]:
  #states are puzzles
  #initial is input
  #successor is toggle
  #final is  when is_solved() = True
  initial = puzzle.board.tobytes()# for uniqueness
  astarqueue = []
  chiz = setayi(heuristic(puzzle)+0, puzzle, [])


  '''chiz.adad= heuristic(puzzle)+0
  chiz.puz= puzzle
  chiz.pat = []'''
  heapq.heappush(astarqueue, chiz)
  visited = {initial}
  nodes_visited = 0

  while not len(astarqueue)==0:
    #temp, (prossesing_node, path) = heapq.heappop(astarqueue)
    #print(heapq.heappop(astarqueue))
    x = heapq.heappop(astarqueue)
    #estimated_cost, prossesing_node, path = heapq.heappop(astarqueue)
    prossesing_node = x.puz
    path = x.pat
    #print('pro')
    #print(prossesing_node)

    nodes_visited = nodes_visited + 1
    if prossesing_node.is_solved():
      return path, nodes_visited
    else:
      for move in prossesing_node.get_moves():
              new_node = deepcopy(prossesing_node)
              new_node.toggle(move[0], move[1])
              visit = new_node.board.tobytes()
              if visit not in visited:
                  visited.add(visit)
                  newcost= len(path)+1+heuristic(new_node)
                  chiz = setayi(newcost, new_node, path + [move])
                  # توجه کنید که هدف ذخیره کردن 
                  #g(n)+h(n)
                  #ها به عنوان مقدار است بنابراین به ازای هر نودی که اضافه میکنیم باید این میزان تا حال ذخیره شود
                  #کاری که newcost
                  #میکند




                  '''chiz.adad= newcost
                  chiz.puz= new_node
                  chiz.pat = path + [move]'''
                  heapq.heappush(astarqueue, chiz)

  return None, nodes_visited



تابع ابتکاری اول بهینگی را تضمین نمیکند بر خلاف دومی

In [20]:
# TODO: Implement heuristic functions and store them in the list

def heuristic1(puzzle: LightsOutPuzzle):
  temp = 0
  for i in range(puzzle.size):
    for j in range(puzzle.size):
      if puzzle.board[i][j] == 1:
        temp+=1
  return temp

def heuristic2(puzzle: LightsOutPuzzle):
  temp = 0
  for i in range(puzzle.size):
    for j in range(puzzle.size):
      if puzzle.board[i][j] == 1:
        temp+=1
  return temp/5

heuristics = [heuristic1, heuristic2]

در ادامه توابع وزن دار هم محاسبه شده اند. 

In [21]:
# TODO: Implement weighted heuristic functions and store them in the list
def weighted_heuristic1(puzzle: LightsOutPuzzle, alpha = 5):
  return heuristic1(puzzle) * alpha

def weighted_heuristic2(puzzle: LightsOutPuzzle, alpha = 2):
  return heuristic2(puzzle) * alpha

weighted_heuristics = [weighted_heuristic1, weighted_heuristic2]

# TODO: assign 2 weights you wanna run
weights = [2, 5]

In [22]:
class SearchStatus(Enum):
    SOLVED = "Solved"
    TIMEOUT = "Timeout"
    NO_SOLUTION = "No Solution"

@dataclass
class SearchResult:
    result: SearchStatus
    solution: Any = None
    nodes_visited: int = None
    time: float = None

    def __str__(self):
        output = f"Result: {self.result.value}\n"
        if self.result == SearchStatus.SOLVED:
            output += f"Solution: {self.solution}\n"
        if self.result != SearchStatus.TIMEOUT:
            output += f"Nodes Visited: {self.nodes_visited}\n"
            output += f"Time Taken: {self.time:.3f} seconds\n"

        return output


DEFAULT_SEARCH_RESULT = SearchResult(SearchStatus.TIMEOUT, None, None, float("inf"))

In [23]:
def run_searches(func: Callable, args: Any, time_limit: float = 60.0, name: str = "") -> SearchResult:
    result = []

    def target():
        try:
            solution, nodes_visited = func(*args)
            result.append((solution, nodes_visited))
        except Exception as e:
            result.append(e)

    thread = threading.Thread(target=target)
    thread.start()

    start_time = time()
    thread.join(timeout=time_limit)

    if thread.is_alive() or not result:
        print(f"\nTime limit of {time_limit} seconds exceeded for {name}")
        return SearchResult(SearchStatus.TIMEOUT)

    if isinstance(result[0], Exception):
        raise result[0]

    solution, nodes_visited = result[0]
    show_solution(args[0], solution, name, nodes_visited)
    time_taken = time() - start_time
    return SearchResult(SearchStatus.SOLVED if solution else SearchStatus.NO_SOLUTION, solution, nodes_visited, time_taken)

In [24]:
def run_test(test: List, heuristics: List[Callable[[Any], int]], time_limit: float = 60.0):

    print(f"Running test:\n {test}")

    puzzle = LightsOutPuzzle(test)

    bfs_result = run_searches(bfs_solve, (deepcopy(puzzle),), name="BFS", time_limit=time_limit)
    ids_result = run_searches(ids_solve, (deepcopy(puzzle),), name="IDS", time_limit=time_limit)

    astar_results = []
    for heuristic in heuristics:
        astar_results.append(
            run_searches(
                astar_solve,
                (deepcopy(puzzle), heuristic),
                name=f"A*({heuristic.__name__})",
                time_limit=time_limit,
            )
        )

    weighted_astar_results = []
    for heuristic in weighted_heuristics:
        for weight in weights:
          weighted_astar_results.append(
              run_searches(
                  astar_solve,
                  (deepcopy(puzzle), partial(heuristic, alpha = weight)),
                  name=f"weighted A*({heuristic.__name__}, alpha = {weight})",
                  time_limit=time_limit,
              )
          )

    print("\n-------------------------------\n")

    return (
        test,
        bfs_result,
        ids_result,
        *astar_results,
        *weighted_astar_results
    )

محدودیت زمانی برای هر تست 180 ثانیه است

In [25]:
results: List[Tuple[str, SearchResult, SearchResult, SearchResult, SearchResult]] = []

for test in tests:
    results.append(run_test(test, heuristics, 180))

Running test:
 [[1 1 1]
 [1 1 1]
 [1 0 0]]

Solving with BFS:
Solution: [(0, 2), (1, 0)]
Nodes visited: 26

Solving with IDS:
Solution: [(0, 2), (1, 0)]
Nodes visited: 37

Solving with A*(heuristic1):
Solution: [(1, 0), (0, 2)]
Nodes visited: 4

Solving with A*(heuristic2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 8

Solving with weighted A*(weighted_heuristic1, alpha = 2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 6

Solving with weighted A*(weighted_heuristic1, alpha = 5):
Solution: [(1, 0), (0, 2)]
Nodes visited: 6

Solving with weighted A*(weighted_heuristic2, alpha = 2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 4

Solving with weighted A*(weighted_heuristic2, alpha = 5):
Solution: [(1, 0), (0, 2)]
Nodes visited: 4

-------------------------------

Running test:
 [[1 1 0]
 [1 0 0]
 [1 1 1]]

Solving with BFS:
Solution: [(0, 0), (0, 1), (1, 0), (1, 1), (1, 2)]
Nodes visited: 272

Solving with IDS:
Solution: [(0, 0), (1, 1), (1, 2), (0, 1), (1, 0)]
Nodes visited: 673

Solving with 

In [26]:
# @title Default title text
table_data: List[List[str]] = []

for result in results:
    table_data.append(
        [
            "\n".join(" ".join(str(cell) for cell in row) for row in result[0]),
            str(result[1]),
            str(result[2]),
        ]
    )

    for i in range(len(heuristics)):
        table_data[-1].append(str(result[2 + i]))

    for i in range(len(weighted_heuristics)):
        for j in range(len(weights)):
          table_data[-1].append(str(result[2 + len(heuristics) + i * 2 + j]))

    table_data.append("\n\n\n")

if table_data[-1] == "\n\n\n":
    table_data.pop()

print(len(table_data))
print("Results:")
print(
    tabulate(
        table_data,
        headers=[
            "Test",
            "BFS",
            "IDS",
            *[f"A* ({heuristic.__name__})" for heuristic in heuristics],
            *[f"Weighted A* ({heuristic.__name__}, alpha = {weight})" for heuristic in weighted_heuristics for weight in weights]
        ],
        floatfmt=".3f",
        numalign="center",
        stralign="center",
        tablefmt="grid",
    )
)

17
Results:
+-----------+----------------------------------------------------+--------------------------------------------------------------------+--------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------+
|   Test    |                        BFS                         |                                IDS                                 |                          A* (heuristic1)                           |                                                  A* (heur